In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('retail_train.csv')


In [3]:
data.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


In [5]:
data['user_id'].nunique()

2499

In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [7]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']

In [8]:
result_test.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [9]:
# making train DataFrame with users and their real purchases
result_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']

In [10]:
result_train.head(5)

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."
2,3,"[866211, 878996, 882830, 904360, 921345, 93194..."
3,4,"[836163, 857849, 877523, 878909, 883932, 89142..."
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
### - Можно сэмплировать товары случайно, но пропорционально какому-либо весу

two kinds of weights: calculated on 'sales_value' and on 'quantity'


In [11]:
# df is dataframe with a column 'item_id' and relevant event column ('quantity' or 'sales_value')
def get_probability(df):
    items = df.item_id
    event_series = df[(df.columns[1])]
    weight_series = np.around(event_series/event_series.sum(), decimals=9)
    
# here should be Exception handling
    if np.around(weight_series.sum())==1:
        # to make the sum of all weights = 1
        remainder = weight_series.sum() - 1     
        weight_series[0] = weight_series[0] - remainder
       
        df['item_weight'] = weight_series
# return df with a new column 'item_weight' containing event-derived probability for each item
        return df
    else:
        return np.nan

In [12]:
def weighted_random_recommendation(df, n=5):
   # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = df.item_id
    item_p = df.item_weight
    
    recs = np.random.choice(items, size=n, replace=False, p=item_p)  
   
    return recs.tolist()

In [13]:
# getting new df with a column 'item_id' and relevant event column 'sales_value'
item_sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_sales.head(2)

,item_id,sales_value
0,25671,20.94
1,26081,0.99


In [14]:
len(item_sales)

86865

In [15]:
# new df with a column 'item_id' and relevant event column 'quantity'
item_quantity = data_train.groupby('item_id')['quantity'].sum().reset_index()
item_quantity.head(2)

,item_id,quantity
0,25671,6
1,26081,1


In [16]:
#getting weights from 'sales_value'
item_sales = get_probability(item_sales)
item_sales.head(2)

,item_id,sales_value,item_weight
0,25671,20.94,1.274000e-06
1,26081,0.99,1.400000e-07


In [17]:
#getting weights from 'quantity'
item_quantity = get_probability(item_quantity)
item_quantity.head(2)

,item_id,quantity,item_weight
0,25671,6,8.348000e-06
1,26081,1,4.000000e-09


Getting recommendations on test data by random sampling with weights from 'sales_value' column 

In [18]:
%%time

# your_code
result_test['rand_rec_sales'] = result_test['user_id'].apply(lambda x: weighted_random_recommendation(item_sales, n=5))
result_test.head(5)

Wall time: 2.76 s


,user_id,actual,rand_rec_sales
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[913009, 1062860, 1025404, 9297055, 969672]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[951190, 882138, 1010950, 843248, 996028]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[906844, 982469, 863447, 968417, 1070820]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[908318, 923149, 874972, 9526344, 1029688]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6534178, 916122, 6534166, 6704062, 1071939]"


Getting recommendations on test data by random sampling with weights from 'quantity' column 

In [19]:
%%time

# your_code
result_test['rand_rec_quant'] = result_test['user_id'].apply(lambda x: weighted_random_recommendation(item_quantity, n=5))
result_test.head(5)

Wall time: 4.85 s


,user_id,actual,rand_rec_sales,rand_rec_quant
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[913009, 1062860, 1025404, 9297055, 969672]","[6534178, 1404121, 6410462, 6533889, 397896]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[951190, 882138, 1010950, 843248, 996028]","[6534178, 6534166, 837677, 2848087, 6544236]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[906844, 982469, 863447, 968417, 1070820]","[6534178, 6533889, 1426702, 6534166, 12389848]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[908318, 923149, 874972, 9526344, 1029688]","[545926, 6534178, 13842139, 6544236, 6533889]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6534178, 916122, 6534166, 6704062, 1071939]","[6534178, 397896, 1404121, 6533889, 6544236]"


In [20]:
#being not ML algorithm, the weighted random recommendation can be tried on train data, too

result_train['rand_rec_sales'] = result_train['user_id'].apply(lambda x: weighted_random_recommendation(item_sales, n=5))

In [21]:
result_train['rand_rec_quant'] = result_train['user_id'].apply(lambda x: weighted_random_recommendation(item_quantity, n=5))

In [22]:
result_train.head(5)

,user_id,actual,rand_rec_sales,rand_rec_quant
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[15596018, 1051791, 879280, 9677270, 990055]","[1102829, 6533889, 6534178, 1404121, 6534166]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[1117813, 1121808, 1070538, 995965, 850652]","[6534178, 6533889, 6544236, 6534166, 1043035]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93194...","[949321, 873178, 9420179, 1104195, 1087102]","[6533889, 6534178, 6534166, 6544236, 397896]"
3,4,"[836163, 857849, 877523, 878909, 883932, 89142...","[819747, 1109206, 12731434, 1133018, 6773196]","[6534178, 6533889, 5747420, 6534166, 842434]"
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69...","[14106445, 898466, 1049788, 8068635, 862682]","[6534166, 6534178, 6533889, 6544236, 1404121]"


### - Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [23]:
# df is dataframe with a column 'item_id' and relevant event column ('quantity' or 'sales_value')
def get_log_weights(df):
    items = df.item_id
    event_series = df[(df.columns[1])]
    weight_series = event_series.apply(lambda x: np.log(x) if x > 1 else 0)
    weight_series = np.around(event_series/event_series.sum(), decimals=12)
    
# here should be Exception handling
    if np.around(weight_series.sum())==1:
        # to make the sum of all weights = 1
        remainder = weight_series.sum() - 1     
        weight_series[0] = weight_series[0] - remainder
       
        df['log_weight'] = weight_series
# return df with a new column 'item_weight' containing event-derived probability for each item
        return df
    else:
        return np.nan

In [24]:
def weighted_log_recommendation(df, n=5):
      
    items = df.item_id
    item_p = df.log_weight
    
    recs = np.random.choice(items, size=n, replace=False, p=item_p)  
   
    return recs.tolist()

In [25]:
item_sales = get_log_weights(item_sales)
item_sales.tail(2)

,item_id,sales_value,item_weight,log_weight
86863,17827644,2.50,3.550000e-07,3.545000e-07
86864,17829232,11.98,1.699000e-06,1.698766e-06


In [26]:
%%time


result_test['log_rec_sales'] = result_test['user_id'].apply(lambda x: weighted_log_recommendation(item_sales, n=5))
result_test.head(5)

Wall time: 2.78 s


,user_id,actual,rand_rec_sales,rand_rec_quant,log_rec_sales
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[913009, 1062860, 1025404, 9297055, 969672]","[6534178, 1404121, 6410462, 6533889, 397896]","[6534178, 12172409, 1137507, 1051884, 904362]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[951190, 882138, 1010950, 843248, 996028]","[6534178, 6534166, 837677, 2848087, 6544236]","[1101502, 937882, 916122, 6534178, 13916186]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[906844, 982469, 863447, 968417, 1070820]","[6534178, 6533889, 1426702, 6534166, 12389848]","[1090002, 6979427, 897954, 1076769, 836445]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[908318, 923149, 874972, 9526344, 1029688]","[545926, 6534178, 13842139, 6544236, 6533889]","[6773219, 1121656, 878996, 9487304, 1126899]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6534178, 916122, 6534166, 6704062, 1071939]","[6534178, 397896, 1404121, 6533889, 6544236]","[918676, 998119, 1118787, 12263590, 1028816]"


In [27]:
item_quantity = get_log_weights(item_quantity)
item_quantity.head(2)

,item_id,quantity,item_weight,log_weight
0,25671,6,8.348000e-06,2.010200e-08
1,26081,1,4.000000e-09,4.362000e-09


In [28]:
result_test['log_rec_quant'] = result_test['user_id'].apply(lambda x: weighted_log_recommendation(item_quantity, n=5))
result_test.head(5)

,user_id,actual,rand_rec_sales,rand_rec_quant,log_rec_sales,log_rec_quant
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[913009, 1062860, 1025404, 9297055, 969672]","[6534178, 1404121, 6410462, 6533889, 397896]","[6534178, 12172409, 1137507, 1051884, 904362]","[6534178, 831888, 6533889, 6544236, 397896]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[951190, 882138, 1010950, 843248, 996028]","[6534178, 6534166, 837677, 2848087, 6544236]","[1101502, 937882, 916122, 6534178, 13916186]","[6534178, 6534166, 6533889, 835004, 397896]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[906844, 982469, 863447, 968417, 1070820]","[6534178, 6533889, 1426702, 6534166, 12389848]","[1090002, 6979427, 897954, 1076769, 836445]","[6534166, 6534178, 1426702, 6533889, 922096]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[908318, 923149, 874972, 9526344, 1029688]","[545926, 6534178, 13842139, 6544236, 6533889]","[6773219, 1121656, 878996, 9487304, 1126899]","[6534178, 6533889, 6534166, 1404121, 6544236]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6534178, 916122, 6534166, 6704062, 1071939]","[6534178, 397896, 1404121, 6533889, 6544236]","[918676, 998119, 1118787, 12263590, 1028816]","[6534178, 6533889, 397896, 6534166, 1426702]"


In [29]:
result_train['log_rec_sales'] = result_train['user_id'].apply(lambda x: weighted_log_recommendation(item_sales, n=5))
result_train.tail(5)

,user_id,actual,rand_rec_sales,rand_rec_quant,log_rec_sales
2494,2496,"[840361, 852159, 871756, 886703, 899624, 91612...","[977186, 826385, 835236, 1078612, 930385]","[6544236, 6534178, 6534166, 6533889, 878398]","[973861, 874308, 834117, 852856, 953609]"
2495,2497,"[838220, 1037840, 1052294, 5569230, 8090537, 1...","[1098248, 1014565, 1067787, 6534178, 6552032]","[6534178, 12949639, 6534166, 6533889, 9575181]","[1018184, 987838, 964594, 890909, 5587502]"
2496,2498,"[824555, 835576, 901776, 904023, 911215, 91749...","[12130728, 837751, 1047595, 1030981, 6904613]","[6534178, 6533889, 1404121, 6534166, 1112473]","[1046587, 1068957, 853172, 898554, 866211]"
2497,2499,"[838186, 853197, 864143, 883665, 932949, 93383...","[849466, 12385624, 1125382, 832937, 7024990]","[6534178, 6533889, 6544236, 6534166, 6410462]","[930870, 835300, 1135044, 6534178, 1109981]"
2498,2500,"[856455, 903476, 834102, 837751, 880797, 89301...","[6442531, 1098066, 944836, 847614, 13986393]","[6533889, 6534178, 6534166, 6544236, 480014]","[1007279, 975085, 7409698, 995242, 938974]"


In [30]:
result_train['log_rec_quant'] = result_train['user_id'].apply(lambda x: weighted_log_recommendation(item_quantity, n=5))
result_train.tail(5)

,user_id,actual,rand_rec_sales,rand_rec_quant,log_rec_sales,log_rec_quant
2494,2496,"[840361, 852159, 871756, 886703, 899624, 91612...","[977186, 826385, 835236, 1078612, 930385]","[6544236, 6534178, 6534166, 6533889, 878398]","[973861, 874308, 834117, 852856, 953609]","[6534178, 6533889, 397896, 6533765, 6534166]"
2495,2497,"[838220, 1037840, 1052294, 5569230, 8090537, 1...","[1098248, 1014565, 1067787, 6534178, 6552032]","[6534178, 12949639, 6534166, 6533889, 9575181]","[1018184, 987838, 964594, 890909, 5587502]","[6534178, 6533889, 6544236, 6534166, 1404121]"
2496,2498,"[824555, 835576, 901776, 904023, 911215, 91749...","[12130728, 837751, 1047595, 1030981, 6904613]","[6534178, 6533889, 1404121, 6534166, 1112473]","[1046587, 1068957, 853172, 898554, 866211]","[6544236, 6534178, 6533889, 1404121, 6534166]"
2497,2499,"[838186, 853197, 864143, 883665, 932949, 93383...","[849466, 12385624, 1125382, 832937, 7024990]","[6534178, 6533889, 6544236, 6534166, 6410462]","[930870, 835300, 1135044, 6534178, 1109981]","[6534178, 480014, 6534166, 5668996, 6533889]"
2498,2500,"[856455, 903476, 834102, 837751, 880797, 89301...","[6442531, 1098066, 944836, 847614, 13986393]","[6533889, 6534178, 6534166, 6544236, 480014]","[1007279, 975085, 7409698, 995242, 938974]","[6534178, 6533889, 397896, 6534166, 964773]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [31]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #assuming top-k items are in the beginning of the list
    flags = np.isin(bought_list, recommended_list[:k])
    
    precision = flags.sum() / k
    
    
    return precision

In [32]:
# evaluating recommendations on train data

In [33]:
result_test.apply(lambda row: precision_at_k(row['rand_rec_sales'], row['actual']), axis=1).mean()

0.02144955925563173

In [34]:
result_test.apply(lambda row: precision_at_k(row['rand_rec_quant'], row['actual']), axis=1).mean()

0.04681684622918707

In [35]:
# evaluating recommendations on train data

In [36]:
result_train.apply(lambda row: precision_at_k(row['rand_rec_sales'], row['actual']), axis=1).mean()

0.09827931172468989

In [37]:
result_train.apply(lambda row: precision_at_k(row['rand_rec_quant'], row['actual']), axis=1).mean()

0.14197679071628652

In [38]:
# with log weights on test and train

In [39]:
result_test.apply(lambda row: precision_at_k(row['log_rec_sales'], row['actual']), axis=1).mean()

0.025465230166503428

In [40]:
result_test.apply(lambda row: precision_at_k(row['log_rec_quant'], row['actual']), axis=1).mean()

0.04720861900097943

In [41]:
result_train.apply(lambda row: precision_at_k(row['log_rec_sales'], row['actual']), axis=1).mean()

0.09603841536614646

In [42]:
result_train.apply(lambda row: precision_at_k(row['log_rec_quant'], row['actual']), axis=1).mean()

0.14205682272909165

### Conclusion
Quantity column seems to be a better weight provider, than sales_value column.
They both seem to be better, than random recommendation without weights (lesson1 precision@5 = 0.0009794319294809011),   
but worse than popular recommendation and ML algorithms

Interesting - worked better on train, though I didn't expect it.

Logs gave almost same results

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [43]:
# your_code
